In [10]:
BUCKET_list = !gcloud info --format='value(config.project)'
BUCKET=BUCKET_list[0]
#BUCKET='qwicklabs-gcp-ffc84680e86718f5'
!bq --location=US mk sparktobq 
!bq --location=US load --autodetect --source_format=CSV sparktobq.kdd_cup_raw gs://$BUCKET/kddcup.data_10_percent.gz

    

BigQuery error in mk operation: Dataset
'qwiklabs-gcp-576b6b9a967fad32:sparktobq' already exists.
BigQuery error in load operation: Error processing job
'qwiklabs-gcp-576b6b9a967fad32:bqjob_r4e97555ee2c385e3_0000016c4c6cc021_1': Not
found: URI gs://qwiklabs-gcp-576b6b9a967fad32


In [11]:
%%bigquery
SELECT * FROM sparktobq.kdd_cup_raw LIMIT 5

Executing query with job ID: d0ed8cc1-f916-42be-b0b5-23df5a57690f
Query executing: 0.23s

NotFound: 404 Not found: Table qwiklabs-gcp-576b6b9a967fad32:sparktobq.kdd_cup_raw was not found in location US

In [12]:
%%bigquery

CREATE OR REPLACE TABLE sparktobq.kdd_cup AS

SELECT 
    int64_field_0 AS duration,
    string_field_1 AS protocol_type,
    string_field_2 AS service,
    string_field_3 AS flag,
    int64_field_4 AS src_bytes,
    int64_field_5 AS dst_bytes,
    int64_field_6 AS wrong_fragment,
    int64_field_7 AS urgent,
    int64_field_8 AS hot,
    int64_field_9 AS num_failed_logins,
    int64_field_11 AS num_compromised,
    int64_field_13 AS su_attempted,
    int64_field_14 AS num_root,
    int64_field_15 AS num_file_creations,
    string_field_41 AS label
FROM
    sparktobq.kdd_cup_raw

Executing query with job ID: 4f85818c-7231-4f3f-af56-817bd863abb8
Query executing: 0.45s

NotFound: 404 Not found: Table qwiklabs-gcp-576b6b9a967fad32:sparktobq.kdd_cup_raw was not found in location US

In [13]:
%%bigquery
SELECT * FROM sparktobq.kdd_cup LIMIT 5

Executing query with job ID: 71025729-80f7-469c-8769-21b2416fdda8
Query executing: 0.24s

NotFound: 404 Not found: Table qwiklabs-gcp-576b6b9a967fad32:sparktobq.kdd_cup was not found in location US

In [14]:
# Create a new Query
%%bigquery connections_by_protocol
SELECT COUNT(*) AS count
FROM sparktobq.kdd_cup
GROUP BY protocol_type
ORDER by count ASC

SyntaxError: invalid syntax (<ipython-input-14-16eccc451d13>, line 3)

In [15]:
# bigQuery equivalent to PySpark from exercise before
%%bigquery attack_stats
    SELECT 
        protocol_type, 
        CASE label
            WHEN 'normal.' THEN 'no attack'
            ELSE 'attack'
        END AS state,
        COUNT(*) as total_freq,
        ROUND(AVG(src_bytes), 2) as mean_src_bytes,
        ROUND(AVG(dst_bytes), 2) as mean_dst_bytes,
        ROUND(AVG(duration), 2) as mean_duration,
        SUM(num_failed_logins) as total_failed_logins,
        SUM(num_compromised) as total_compromised,
        SUM(num_file_creations) as total_file_creations,
        SUM(su_attempted) as total_root_attempts,
        SUM(num_root) as total_root_acceses
    FROM sparktobq.kdd_cup
    GROUP BY protocol_type, state
    ORDER BY 3 DESC

IndentationError: unexpected indent (<ipython-input-15-c9214de79c1a>, line 3)

In [16]:
%matplotlib inline
ax = attack_stats.plot.bar(x='protocol_type', subplots=True, figsize=(10,25))

NameError: name 'attack_stats' is not defined

In [17]:
import google.cloud.storage as gcs

# save locally
ax[0].get_figure().savefig('report.png');
connections_by_protocol.to_csv("connections_by_protocol.csv")

# upload to GCS
bucket = gcs.Client().get_bucket(BUCKET)
for blob in bucket.list_blobs(prefix='sparktobq/'):
    blob.delete()
for fname in ['report.png', 'connections_by_protocol.csv']:
    bucket.blob('sparktobq/{}'.format(fname)).upload_from_filename(fname)

NameError: name 'ax' is not defined